In [1]:
# importing the necessary libraries
import requests
import pymongo
import datetime
from pymongo import MongoClient

In [2]:
API_KEY = '392b8af66ad74d929a891019253101' # API key for accessing the weather service (replace with your actual key)
CITY = 'Lagos'
MONGO_URI = 'mongodb://127.0.0.1:27017/'  # MongoDB connection URI (local database in this case)
DB_NAME = 'weather_db'
COLLECTION_NAME = 'weather_data'


In [5]:
# Step 1: Extract the data
def extract_weather_data():
    url = f'https://api.weatherapi.com/v1/forecast.json?key={API_KEY}&q={CITY}&days=7'
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()  # Converting the response to JSON
    else:
        print(f"Error fetching weather data: {response.status_code}")
        return None

In [7]:
# Step 2: Transform the data
def transform_weather_data(weather_data):
    if not weather_data:
        print("No weather data provided.")
        return None

    today = datetime.datetime.now().date()
    favorable_days = []

    # Check forecast for the next 7 days
    for forecast in weather_data.get('forecast', {}).get('forecastday', []):
        forecast_date = datetime.datetime.strptime(forecast['date'], '%Y-%m-%d').date()
        
        if forecast_date > today:  # Ensure we are checking next week's forecast
            temperature = forecast['day']['avgtemp_c']
            rain = forecast['day']['totalprecip_mm']

            favorable = 20 <= temperature <= 30 and rain == 0  # Favorable conditions
            
            favorable_days.append({
                'date': forecast['date'],
                'favorable': favorable,
                'temperature': temperature,
                'rain': rain
            })

    return favorable_days

# Fetch weather data
weather_data = extract_weather_data()

# Process weather data
result = transform_weather_data(weather_data)
print(result)

[{'date': '2025-03-30', 'favorable': False, 'temperature': 29.3, 'rain': 0.52}, {'date': '2025-03-31', 'favorable': False, 'temperature': 29.3, 'rain': 1.49}]


In [ ]:
# Step 3: Load - Store the processed data in MongoDB
def load_to_mongodb(data):
    if not data:
        print("No data to load into MongoDB.")
        return

    client = MongoClient(MONGO_URI)
    db = client[DB_NAME]
    collection = db[COLLECTION_NAME]

    # Insert the data into MongoDB
    collection.insert_many(data)
    print("Data successfully loaded into MongoDB.")

# Run the ETL Process
def run_etl():
    print("Starting ETL Process...")
    
    # Extract
    weather_data = extract_weather_data()
    
    # Transform
    transformed_data = transform_weather_data(weather_data)
    
    # Load
    load_to_mongodb(transformed_data)
    
    print("ETL Process completed.")

# Execute the ETL process
if __name__ == '__main__':
    run_etl()


